# Técnica de prompt tuning

En este cuaderno se mostrará como realizar un instruction prompt a un gran modelo de lenguaje (LLM), concretamente, al modelo [FLAN-T5](https://huggingface.co/docs/transformers/model_doc/flan-t5) creado por Google. Elegiremos en este caso su [versión base](https://huggingface.co/google/flan-t5-base).

Mediante esta técnica, utilizaremos prompts para ajustar el modelo usando PEFT (Prompt Enhanced Fine-Tuning) y así mejorar su rendimiento en tareas específicas.

Al utilizar Prompt Tuning con PEFT (Parameter-Efficient Fine-Tuning) en un modelo como FLAN-T5, el modelo original permanece congelado, es decir, sus pesos preentrenados no se actualizan. En su lugar, se entrenan únicamente los parámetros de los prompts suaves (soft prompts), que son embeddings aprendibles que se añaden al input del modelo. Esta técnica permite ajustar el modelo de manera eficiente para tareas específicas sin necesidad de modificar todos sus parámetros.

Veremo como, al imprimir los parámetros del modelo, solo reentrenaremos una pequeña fracción de los mismos, menor al 1% de los parámetros totales del modelo.

Además, no lo veremos en este cuaderno, los modelos se pueden cuantizar (utilizando, por ejemplo, LoRa junto con PEFT) para reducir aún más el tamaño del modelo y hacerlo más eficiente.

## Paso 1: Instalación e importación de librerías y definición de parámetros

Se instalan a continuación las librerías necesarias para la ejecución de este cuaderno.

In [1]:
!pip install wandb accelerate peft tqdm==4.66.1 datasets==2.15.0 transformers==4.46.2
'''
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
'''

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 

'\n!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118\n'

In [2]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset, Dataset
from tqdm import tqdm
from peft import get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit

Se importan las librerías necesarias para este cuaderno.

Una vez importadas, se definen una serie de parámetros que se utilizarán a lo largo del cuaderno.

- **model_name**: Nombre del modelo que será reentrenado.
- **dataset_name**: Nombre del dataset utilizado.
- **epochs**: Número de épocas de entrenamiento.
- **max_length**: Longitud máxima de las secuencias de entrada. El máximo del modelo son 512.
- **use_cuda**: Booleano que indica si se utilizará la GPU para entrenar.
- **results_dir**: Directorio donde se guardarán los resultados.
- **num_virtual_tokens**: Número de tokens virtuales que se añadirán al vocabulario del modelo. Los tokens virtuales son embeddings que se añaden al vocabulario del modelo y que se utilizan para mejorar el rendimiento del modelo en tareas específicas. Se entrenan durante el proceso de prompt tuning.

In [51]:
args = {}
args['model_name'] = "google/flan-t5-base"
args['dataset_name'] = "fancyzhx/ag_news"
args['epochs'] = 2
args['max_length'] = 300
args['use_cuda'] = True
args['results_dir'] = "experiments/test"
args['num_virtual_tokens'] = 20

## Paso 2: Carga y procesado del corpus

Mediante la función `load_dataset` de la librería `datasets`, se puede cargar un dataset de manera sencilla que esté publicado en HuggingFace.

En este caso se va cargar el dataset `fancyzhx/ag_news` que contiene más de 100.000 noticias en inglés clasificadase en 4 clsaes: Sci/Tech, Sports, Business y World. Toda la información se encuentra en [este enlace](https://huggingface.co/datasets/fancyzhx/ag_news).

In [4]:
dataset = load_dataset(args['dataset_name'])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Este dataset contiene los datos divididos en 2 particiones: `train` y `test`. Como originalmente no contiene una partición de validación, se creará una extrayendo 3.00 muestras del conjunto de entrenamiento. Se utilizará la partición de `train` para entrenar el modelo, la de `validation` para validar el modelo y la de `test` para evaluar el modelo.

In [5]:
#Extract 3000 sample from the train set to create the validation set
newSplits = dataset['train'].train_test_split(test_size=3000, shuffle=True)

dataset['train'] = newSplits['train']
dataset['validation'] = newSplits['test']

Las columnas que contiene el dataset son las siguientes:

- **text**: Texto de la noticia.
- **label**: Clase de la noticia (0, 1, 2 o 3).

Se define la función `add_label_column` que se encargará de transformar el número de la clase en una etiqueta legible (Sci/Tech, Sports, Business o World). Para ello, se definen los diccionarios `label2name` y `name2label` que mapean las clases con las etiquetas.

In [6]:
label2id = {
    'World': 0,
    'Sports': 1,
    'Business': 2,
    'Sci/Tech': 3
}

id2label = {v: k for k, v in label2id.items()}

In [7]:
#Append a column in the dataset for the labelText
def add_label_column(example):
    example['labelText'] = id2label[example['label']]
    return example

dataset = dataset.map(add_label_column)

Map:   0%|          | 0/117000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Para agilizar el proceso de entrenamiento, se reducirá el tamaño del dataset a 10.000 muestras para la partición de `train` y a 3.000 muestras para las particiones de `validation` y `test`.

In [8]:
#Get just 5.000 samples for training, 1.500 for validation and 1.500 for testing
dataset['train'] = dataset['train'].select(range(5000))
dataset['validation'] = dataset['validation'].select(range(1500))
dataset['test'] = dataset['test'].select(range(1500))

Una vez tenemos los datos cargados y correctos, se ha definido la función `add_prompt_column`, la cual se encargará de añadir una columna al corpus con el prompt correspondiente a cada noticia. Para ello, se ha definido el siguiente prompt:

```
Given the following text: {text}
Predict its corresponding category (World, Sports, Business, Sci/Tech):
```

Se mapeará cada noticia con su prompt correspondiente y se guardará en una nueva columna llamada `prompt`.

In [12]:
#Create the prompt
def add_prompt_column(example):
    #Replace \ by a space
    example['text'] = example['text'].replace('\\', ' ')
    example['prompt'] = f'Given the following text: {example["text"]}\nPredict its corresponding category (World, Sports, Business, Sci/Tech):'
    return example

dataset = dataset.map(add_prompt_column)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

## Paso 3: Reentrenamiento del modelo

Se define la función `preprocess_function` que se encargará de procesar los datos de entrada para que puedan ser utilizados por el modelo.

Esta función, usando el tokenizador, se tokenizan los prompts. Además, se tokenizan también los textos correspondientes a las categorias de cada una de las noticias.

<span style="color:red">Atención:</span> Esta función está definida para poder ser usada de forma paralela (`batched=True`) al ser usada por la función `map` de la librería `datasets`. Esto permite que el procesado de los datos sea más rápido, al procesar varios elementos a la vez. Es por ello que te pueda parecer que la función está definida de forma extraña.

In [19]:
def preprocess_function(sample, padding="max_length"):

    # Usar el tokenizador para generar los inputs del modelo para cada elemento del dataset, dado el texto
    model_inputs = tokenizer(
        sample['prompt'],
        max_length=args['max_length'],
        padding=padding,
        truncation=True,
    )

    # Usar el tokenizador para generar los targets del modelo para cada elemento del dataset, dado el texto anotado
    target_diseases = tokenizer(
        text_target=sample["labelText"],
        max_length=args['max_length'],
        padding=padding,
        truncation=True,
    )

    # Si se está haciendo padding (es decir, si se está fijando el tamaño máximo de la secuencia de entrada y no considerando el
    # tamaño de cada secuencia de entrada), se reemplazan todos los tokens de padding por -100 para que no se consideren en la
    # función de pérdida. Esto se realiza para que el modelo ignore los tokens de padding en la función de pérdida y no se
    # penalice por ellos, ni se aprenda a predecirlos.
    if padding == "max_length":
        target_diseases["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label]
            for label in target_diseases["input_ids"]
        ]

    # Se agregan los targets al diccionario de inputs del modelo
    model_inputs["labels"] = target_diseases["input_ids"]

    # Se retorna el diccionario de inputs del modelo
    return model_inputs

Se importa el tokenizador y el modelo.

El tokenizador se importa haciendo uso de la función `T5Tokenizer` de la librería `transformers`. Esta función se encarga de cargar el tokenizador para el modelo que se le pasa como parámetro.

Antes de cargar el modelo, se define la configuración de PEFT. En ella, definimos la tarea a realizar (`task_type`), que en este caso es `CAUSAL_LM` (tarea utilizada para generación de texto), la inicialización del prompt tuning `prompt_tuning_init` (siendo una inicialización aleatoria en este caso), el número de tokens virtuales que se añadirán al vocabulario del modelo (`num_virtual_tokens`), y por último se define el nombre del tokenizador (`tokenizer_name_or_path`).

Una vez definida la configuración, ya se puede hacer uso de la función `get_peft_model` de la librería `transformers` para cargar el modelo. Esta función se encarga de cargar el modelo adecuado para la configuración de PEFT que se le pasa como parámetro (la que acabamos de definir).

In [37]:
tokenizer = T5Tokenizer.from_pretrained(args['model_name'], legacy = False)
model = T5ForConditionalGeneration.from_pretrained(args['model_name']).to('cuda')

In [21]:
generation_config_prompt  = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,  # This type indicates the model will generate text.
    prompt_tuning_init=PromptTuningInit.RANDOM,  # The added virtual tokens are initializad with random numbers
    num_virtual_tokens=args['num_virtual_tokens'],  # Number of virtual tokens to be added and trained.
    tokenizer_name_or_path=args['model_name'],  # The pre-trained model.
)

In [22]:
peft_model_prompt = get_peft_model(model, generation_config_prompt)
print(peft_model_prompt.print_trainable_parameters())

trainable params: 15,360 || all params: 247,593,216 || trainable%: 0.0062
None


Como detalle técnico, debemos de activar en los parámetros del modelo que se realice el cálculo de los gradientes para los parámetros introducidos al modelo.

In [23]:
for param in model.parameters():
    param.requires_grad = True

Vemos como el número de parámetros entrenables es del 0.0062%, ¡muy pocos!. Esta diferencia es clave para entender la eficiencia de PEFT y el prompt tuning respecto al instrucción tuning. En este caso, no estamos modificando los pesos del modelo, sino que estamos añadiendo embeddings aprendibles que se utilizan como prompts. Por contra, en el instrucción tuning, se reentrenan todos los pesos del modelo.

Antes de utilizar la función definida, debemos de dilucidar cuál es el tamaño máximo de los textos de las noticias. Para ello, iteraremos sobre el dataset y guardaremos el tamaño máximo de los textos en la variable `max_text_length`. Ajustando el tamaño máximo de los textos, podremos reducir el tiempo de procesado de los datos.

In [50]:
#Calculate the maximum length of the input and output

#Get the text with the maximum length
maxText = dataset['train']['prompt'][0]
for text in dataset['train']['prompt']:
    if len(text) > len(maxText):
        maxText = text

print('** LONGEST PROMPT **')
print(maxText)
print('****')

print("Length of the prompt: ", str(len(maxText)))

#tokenize the text
inputs = tokenizer(
    maxText,
    max_length=5000,
    truncation=True,
)

#Get the maximum length of the input
max_input_length = len(inputs['input_ids'])

print("Max input length: ", max_input_length)

** LONGEST PROMPT **
Given the following text: Baltimore's  quot;Free Books! quot; Charity in Dire Straits I spend anywhere from three to eight hours every week sweating along with a motley crew of local misfits, shelving, sorting, and hauling ton after ton of written matter in a rowhouse basement in Baltimore. We have no heat nor air conditioning, but still, every week, we come and work. Volunteer night is Wednesday, but many of us also work on the weekends, when we're open to the public. There are times when we're freezing and we have to wear coats and gloves inside, making handling books somewhat tricky; other times, we're all soaked with sweat, since it's 90 degrees out and the basement is thick with bodies. One learns to forget about personal space when working at The Book Thing, since you can scarcely breathe without bumping into someone, and we are all so accustomed to having to scrape by each other that most of us no longer bother to say "excuse me" unless some particularly dra

Una vez se tiene el tokenizador y la función `preprocess_function` definidos, se aplica esa función sobre todo el dataset mediante la función `map` de la librería `datasets`.

In [52]:
tokenized_dataset = dataset.map(
    preprocess_function, batched=True, remove_columns=["text", "labelText", "label", "prompt"]
)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Se definen los hiperparámetros de entrenamiento utilizando la clase `TrainingArguments` de la librería `transformers`. Esta clase permite definir los hiperparámetros de entrenamiento de una manera sencilla. (Más información en [este enlace](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments)).

En este caso se han definido los siguientes hiperparámetros:

- **output_dir**: Directorio donde se guardarán los resultados.
- **auto_find_batch_size**: Booleano que indica si se debe encontrar automáticamente el tamaño de batch más adecuado acorde a los recursos disponibles.
- **learning_rate**: Tasa de aprendizaje. En este caso se utilizará una tasa de aprendizaje de 1e-5.
- **num_train_epochs**: Número de épocas de entrenamiento.

In [53]:
training_args = TrainingArguments(
        output_dir=args['results_dir'],
        auto_find_batch_size=True,
        learning_rate = 0.0035,
        num_train_epochs=1
    )

Se define ahora el _data collator_, que es el encargado de procesar los datos de entrada para que puedan ser utilizados por el modelo. En este caso se utiliza la clase `DataCollatorForLanguageModeling` de la librería `transformers`. Esta clase se encarga de procesar los datos de entrada para que puedan ser utilizados por el modelo T5. (Más información en [este enlace](https://huggingface.co/docs/transformers/main_classes/data_collator#transformers.DataCollatorForLanguageModeling)).

In [55]:
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

Ahora, se define el objeto para reentrenar el modelo, en este caso se utiliza la clase `Trainer` de la librería `transformers`. Esta clase se encarga de reentrenar un modelo T5. (Más información en [este enlace](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Trainer)).

En él se define el modelo a usar, los argumentos de entrenamiento, el _data collator_ y el dataset de entrenamiento y de validación.

In [56]:
trainer = Trainer(
    model=peft_model_prompt,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator
)


Se ejecuta el reentrenamiento del modelo.

<span style="color:red">Atención:</span> Puede ser que te pida una API Key de WandB. WandB (Weights & Biases) es una plataforma integral para el seguimiento y visualización de experimentos de aprendizaje automático. Facilita el registro y comparación de hiperparámetros y métricas, así como la colaboración entre equipos al proporcionar un espacio centralizado para compartir resultados y códigos. Con integración fácil en bibliotecas populares, WandB se destaca por su capacidad para mejorar la eficiencia en la gestión y comprensión de modelos, convirtiéndose en una herramienta valiosa para profesionales de aprendizaje automático.

In [57]:
modelTrainer = trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: egr68 (gplsi_continual) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss


Step,Training Loss
500,0.319200
1000,0.058100


Una vez reentrenado, se guarda el modelo y el tokenizador en el directorio `results_dir`.

In [58]:
trainer.model.save_pretrained(args['results_dir'])

## Paso 4: Inferencia

Para realizar la inferencia, se importa usando `PeftModel` el modelo ya reentrenado y el tokenizador. Se importan a continuación.

In [59]:
from peft import PeftModel

loaded_model = PeftModel.from_pretrained(model,
                                         args['results_dir'],
                                         is_trainable=False)

Se crea una función auxiliar llamada `generateOutput`, en la que dado el modelo, el tokenizador y un prompt, se genera la salida del modelo.

In [60]:
def generateOutput(model, tokenizer, prompt):

    #Se tokeniza el prompt
    inputs = tokenizer(prompt, return_tensors="pt")

    #Se obtiene la salida del modelo, dados:
    # - los inputs tokenizados
    # - la máscara de atención
    # - la longitud máxima de la secuencia de salida, establecida en este caso en 32
    # - el número de beams a usar en la decodificación, establecido en este caso en 5. Un beam es una hipótesis de salida
    #   que el modelo considera como una posible solución al problema. El modelo genera varias hipótesis de salida y
    #   selecciona la mejor de ellas como la salida final.
    # - early_stopping=True para que el modelo deje de generar hipótesis de salida cuando todas las hipótesis generadas
    #   tengan el token de fin de secuencia (</s>) o cuando se haya generado el número máximo de hipótesis de salida
    outputs = model.generate(
        inputs.input_ids.to(model.device),
        attention_mask=inputs.attention_mask.to(model.device),
        max_length=1000,
        num_beams=5,
        early_stopping=True,
        return_dict_in_generate=True
    )
    decoded_output = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)

    return decoded_output

Se puede comprobar con el siguiente código que la inferencia funciona correctamente en un único ejemplo. Puedes probar con otros ejemplos cambiando el valor de i.

Para estos ejemplos, hemos utilizado el conjunto de entrenamiento, por tanto, es normal que el modelo suela acertar la categoría de la noticia.

In [61]:
i = 5
generated = generateOutput(model, tokenizer, dataset['train'][i]['prompt'])
real = dataset['train'][i]['labelText']

print(f"Prompt: {dataset['train'][i]['prompt']}")
print(f"Generated: {generated}")
print(f"Real: {real}")
print(f"Is the generated text the same as the real one? Yes" if generated == real else "Is the generated text the same as the real one? No")

From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.


Prompt: Given the following text: Rwanda Retains Right to Attack Rebels in Congo (Reuters) Reuters - Rwanda can and will strike Rwandan rebels in Congo if little continues to be done to disarm them, but Kigali is not spoiling for a fight, President Paul Kagame said on Saturday after meeting his Congolese counterpart.
Predict its corresponding category (World, Sports, Business, Sci/Tech):
Generated: World
Real: World
Is the generated text the same as the real one? Yes


Se define una función llamada `getMatchMetricBatch` para que realice una métrica que simplemente mida las coincidencias exactas entre la salida del modelo y la categoría real con respecto al total.

Para agilizar el proceso, se realizará la inferencia por lotes (batch). Para ello, se define la función `generateOutputBatch`, la cual se encargará de generar la salida del modelo para un lote de ejemplos.

In [62]:
def generateOutputBatch(model, tokenizer, prompts, batch_size=32):

    outputs = []
    for i in range(0, len(prompts), batch_size):
        # Select a batch of prompts
        batch_prompts = prompts[i:i + batch_size]

        # Tokenize the batch
        inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True, truncation=True)

        # Move tensors to the same device as the model
        inputs = {key: tensor.to(model.device) for key, tensor in inputs.items()}

        # Generate outputs for the batch
        batch_outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=50,
            num_beams=5,
            early_stopping=True,
            return_dict_in_generate=True
        )

        # Decode the generated sequences and append them to the results
        decoded_outputs = [tokenizer.decode(seq, skip_special_tokens=True) for seq in batch_outputs.sequences]
        outputs.extend(decoded_outputs)

    return outputs

In [63]:
def getMatchMetricBatch(model, tokenizer, split, batch_size=32):
    matches = 0
    total = len(split)

    # Process the dataset in batches
    for i in tqdm(range(0, total, batch_size)):
        # Extract a batch of prompts and labels
        batch = split[i:i + batch_size]
        prompts = batch['prompt']
        true_labels = batch['labelText']

        # Generate outputs for the batch
        generated_outputs = generateOutputBatch(model, tokenizer, prompts, batch_size=batch_size)

        # Compare generated outputs with true labels
        for generated, true_label in zip(generated_outputs, true_labels):
            if generated.strip() == true_label.strip():
                matches += 1

    # Calculate accuracy
    accuracy = matches / total

    return {'matches': matches, 'total': total, 'accuracy': accuracy}

Se ejecutan las métricas y se muestran los resultados.

In [64]:
batch_size = 12

print("Evaluating training set...")
trainMetric = getMatchMetricBatch(model, tokenizer, dataset['train'], batch_size=batch_size)
print(f"Train matches: {trainMetric['matches']}")
print(f"Train accuracy: {trainMetric['accuracy']}")

print("Evaluating validation set...")
validationMetric = getMatchMetricBatch(model, tokenizer, dataset['validation'], batch_size=batch_size)
print(f"Validation matches: {validationMetric['matches']}")
print(f"Validation accuracy: {validationMetric['accuracy']}")

print("Evaluating test set...")
testMetric = getMatchMetricBatch(model, tokenizer, dataset['test'], batch_size=batch_size)
print(f"Test matches: {testMetric['matches']}")
print(f"Test accuracy: {testMetric['accuracy']}")

Evaluating training set...


100%|██████████| 417/417 [02:19<00:00,  2.98it/s]


Train matches: 4408
Train accuracy: 0.8816
Evaluating validation set...


100%|██████████| 125/125 [00:40<00:00,  3.06it/s]


Validation matches: 1340
Validation accuracy: 0.8933333333333333
Evaluating test set...


100%|██████████| 125/125 [00:41<00:00,  3.04it/s]

Test matches: 1315
Test accuracy: 0.8766666666666667


Se puede observar que el modelo ha acertado en la mayoría de los casos, superando en todos los conjuntos el 85% de aciertos.